## Extraktionsbaserad textsammanfattare med olika rankningsmått 

#### Imports

In [23]:
# Imports
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np
import nltk 
import ssl
import re 

# Summarization length of original text
percentage = 0.15

# Fixes some errors, found online at https://github.com/gunthercox/ChatterBot/issues/930#issuecomment-322111087
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

#### Input

In [24]:
# Web scrapping -->  module för att ladda ner artiklar 
from newspaper import Article
text = "https://www.aftonbladet.se/nyheter/a/kE6ExL/sd-far-tunga-poster-i-utskotten"#'https://www.svt.se/nyheter/utrikes/stall-dina-fragor-om-kriget-till-svt-s-utrikesreportrar'
article = Article(text, language='sv')
article.download()
article.parse()
text = article.text

# Beroende på vilken hemsida nyheten kommer ifrån kan titeln och texten inehålla delar av sidan man egentligen inte bryr sig om
# T.ex. från aftonbladet är titeln med i texten och texten innehåller en mening som: "publicerad: 30 sep", man kan ta bort detta men 
# det blir om vi får tid över.

#print('Title:' , article.title, '\n\nText: \n', text)

# Preprocessing 

## Overview
### Calculate number of sentences to keep

### List 1 - sentences
* Varje mening separat

### Dataframe - scores
#### columns are the score of each ranking measure
* Baseline
* Headings
* TF/IDF-score
* NER
* ~~Class~~ //Om vi har tid för ML

### List 2 -> Cleaned for Stop Words 
* Varje mening separat 

###

##### Original Sentences List

In [25]:
# removes endlines:
from token import NEWLINE

org_sentences = text.replace('\n\n', '. ')
# creates some exceptions from above rule
org_sentences = org_sentences.replace('.. ', '. ')
org_sentences = org_sentences.replace(':. ', ': ')
org_sentences = org_sentences.split('. ')

org_sentences[0:5]

['SD får tunga poster i utskotten',
 'Publicerad: 30 september Uppdaterad: 30 september',
 'Sverigedemokraterna får ordförandeposten i riksdagens justitie- och utrikesutskott',
 'Nu går ledarna för vänsterblocket till hård attack',
 '– Det är skrämmande, ganska chockartat, säger Socialdemokraternas gruppledare Lena Hallengren till Aftonbladet']

##### Dataframe

In [26]:
index = range(org_sentences.__len__())
columns = ['Baseline', 'Headings', 'TF', 'NER']
scores = pd.DataFrame(index=index, columns=columns)
scores.fillna(0, inplace=True)
scores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34 entries, 0 to 33
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   Baseline  34 non-null     int64
 1   Headings  34 non-null     int64
 2   TF        34 non-null     int64
 3   NER       34 non-null     int64
dtypes: int64(4)
memory usage: 1.2 KB


#### Create spacy doc object

In [27]:
import spacy
# Credit to Explosion for sv_core_news_sm --> https://github.com/explosion 
# "lemmatization accuracy 0.95"
# Create spacy nlp object 
nlp = spacy.load("sv_core_news_sm") # nlp used by lemmatizer()

#### Lemmatizer

In [28]:
def lemmatizer(list_of_strings):
    lemmatized_sentences = []
    lemmatized_sentence = ''
    for i in range(len(list_of_strings)): 
        sentence_to_lemmatize = nlp(list_of_strings[i])
        for token in sentence_to_lemmatize:
            lemma = token.lemma_
            lemmatized_sentence += lemma + ' '  
        
        lemmatized_sentences.append(lemmatized_sentence)
        lemmatized_sentence = '' 

    return lemmatized_sentences

lemma = lemmatizer(org_sentences)
print(lemma)

['SD få tung post i utskott ', 'publicerad : 30 september uppdaterad : 30 september ', 'Sverigedemokraterna få ordförandepost i riksdag justitie och utrikesutskott ', 'nu gå ledarna för vänsterblock till hård attack ', '– den vara skrämmande , ganska chockarta , säga Socialdemokraterna gruppledare Lena Hallengren till Aftonbladet ', 'Sverigedemokraterna , Moderat , Kristdemokraterna och Liberalerna ha dela upp post i utskott och EU-nämnden ', 'där ta Sverigedemokraterna flera viktig post ', 'bland annan tilldela parti ordförandepost i arbetsmarknadsutskott , näringsutskott , justitieutskott samt utrikesutskottet , enligt en pressmeddelande ', 'de erhåller även post som vice ordförande i civilutskott , trafikutskott , försvarsutskotte samt skatteutskotte ', '– den som överraska jag mycket , men som jag kunna se varför de vilja ha , vara ordförandepost i utrikesutskott ', 'den vara en tecken på att de lycka i förhandling med M och KD , säga Aftonbladet My Rohwedder i Aftonbladet tv ', '”

#### Proper Nouns

In [29]:
def proper_nouns(list_of_strings):
    proper_nouns = []
    for i in range(len(list_of_strings)): 
        sentence_to_pos = nlp(list_of_strings[i])
        for token in sentence_to_pos: 
            token_str = token.text
            if token.pos_ == "PROPN" and len(token_str) > 1:
                proper_nouns.append(token_str)
    return proper_nouns

print(proper_nouns(org_sentences))


['SD', 'Socialdemokraternas', 'Lena', 'Aftonbladet', 'KD', 'Rohwedder', 'Aftonbladet', 'Sveriges', 'Kristerssons', 'Lena', 'Hallengren', 'Socialdemokraterna', 'Sverige', 'Sverige', 'Stenevi', 'Putin', 'Biden', 'Sverige', 'Esbati', 'Martin', 'Kinnunen', 'Markus', 'Wiechels', 'Syrien', 'Wiechel', 'Martin', 'Kinnunen', 'UD', 'Syrien', 'Annie', 'KD', 'SD', 'Sverige', 'Nato', 'EU', 'Sveriges', 'Jakob', 'Hallgren']


#### Named Entities

In [30]:
def named_entity_recognition(list_of_strings):
    doc = nlp(' '.join(list_of_strings))
    # Convert tuple[Span] to str
    named_entities = doc.ents.__str__()
    # Remove string parenthesis 
    named_entities = named_entities[1:len(named_entities) - 1]
    # Create list of strings
    named_entities = named_entities.split(',')
    return named_entities

### Stop Word Filtering

In [31]:
# Inspired by https://www.geeksforgeeks.org/removing-stop-words-nltk-python/
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

def get_swe_stop_words(): 
    swe_stop_words = set(stopwords.words('swedish'))
    swe_stop_words.update([',', '"', ':', '-', '–', '”'])
    return swe_stop_words

def stop_word_filtering(list_of_strings):
    word_tokens = word_tokenize(' '.join(org_sentences))
    filtered_sentence = [w for w in word_tokens if not w.lower() in get_swe_stop_words()]
    return filtered_sentence

#### Frequency Distribution 

In [32]:
def frequency_distribution(list): 
    fdist = FreqDist(word.lower() for word in word_tokenize(' '.join(list)))
    return fdist

In [33]:
# TEsting 
filtered_sentence = stop_word_filtering(org_sentences) 
print(filtered_sentence)

['SD', 'får', 'tunga', 'poster', 'utskotten', 'Publicerad', '30', 'september', 'Uppdaterad', '30', 'september', 'Sverigedemokraterna', 'får', 'ordförandeposten', 'riksdagens', 'justitie-', 'utrikesutskott', 'går', 'ledarna', 'vänsterblocket', 'hård', 'attack', 'skrämmande', 'ganska', 'chockartat', 'säger', 'Socialdemokraternas', 'gruppledare', 'Lena', 'Hallengren', 'Aftonbladet', 'Sverigedemokraterna', 'Moderaterna', 'Kristdemokraterna', 'Liberalerna', 'delat', 'posterna', 'utskotten', 'EU-nämnden', 'tar', 'Sverigedemokraterna', 'flera', 'viktiga', 'poster', 'Bland', 'annat', 'tilldelas', 'partiet', 'ordförandeposten', 'arbetsmarknadsutskottet', 'näringsutskottet', 'justitieutskottet', 'samt', 'utrikesutskottet', 'enligt', 'pressmeddelande', 'erhåller', 'även', 'posten', 'vice', 'ordförande', 'civilutskottet', 'trafikutskottet', 'försvarsutskottet', 'samt', 'skatteutskottet', 'överraskade', 'mest', 'se', 'vill', 'ordförandeposten', 'utrikesutskottet', 'tecken', 'lyckats', 'förhandlinge

# Ranking Measures

### Baseline
(1/N --> n=ordning mening kommer i dvs. första meningen får N=1 -> 1/1, andra meningen får N=2 -> 1/2, osv.)

In [34]:
# Ranking metric 1 --> Baseline
for i, score in enumerate(scores['Baseline']) :
    scores['Baseline'][i] = 1/((i+1))
scores.describe()

,Baseline,Headings,TF,NER
count,34.000000,34.0,34.0,34.0
mean,0.121124,0.0,0.0,0.0
std,0.183991,0.0,0.0,0.0
min,0.029412,0.0,0.0,0.0
25%,0.038846,0.0,0.0,0.0
50%,0.057190,0.0,0.0,0.0
75%,0.108333,0.0,0.0,0.0
max,1.000000,0.0,0.0,0.0


### Headings

In [35]:
# Ranking metric 2 --> Headings
# Sets all 'Headings' scores to 0, mostly for testing so i can run this multiple times, 
# but also to make sure nothing weird has happened earlier in the code.
scores['Headings'] = 0
for i, sentence in enumerate(org_sentences):
    for word in article.title.split(' '):
        if word in sentence:
            scores.at[i, 'Headings'] += 1
scores.describe()

,Baseline,Headings,TF,NER
count,34.000000,34.000000,34.0,34.0
mean,0.121124,1.352941,0.0,0.0
std,0.183991,1.011520,0.0,0.0
min,0.029412,0.000000,0.0,0.0
25%,0.038846,1.000000,0.0,0.0
50%,0.057190,1.000000,0.0,0.0
75%,0.108333,1.000000,0.0,0.0
max,1.000000,6.000000,0.0,0.0


## TERM FREQUENCY 


In [36]:
#
scores['TF'] = 0

fdist = frequency_distribution(org_sentences)

for i, sentence in enumerate(org_sentences):
    for word in sentence.split(' '):
        #word = word.lemma
        if word in fdist.keys():
            scores.at[i, "TF"] += fdist.get(word)

scores.describe()
#scores['TF']
scores.head

<bound method NDFrame.head of     Baseline  Headings   TF  NER
0   1.000000         6   32    0
1   0.500000         1    8    0
2   0.333333         2   53    0
3   0.250000         1   17    0
4   0.200000         1   39    0
5   0.166667         3  101    0
6   0.142857         2    6    0
7   0.125000         1   39    0
8   0.111111         1   48    0
9   0.100000         1  104    0
10  0.090909         1  127    0
11  0.083333         1    3    0
12  0.076923         1    6    0
13  0.071429         0   29    0
14  0.066667         1  150    0
15  0.062500         1   11    0
16  0.058824         1  266    0
17  0.055556         1  144    0
18  0.052632         1  174    0
19  0.050000         1    3    0
20  0.047619         1   22    0
21  0.045455         1   98    0
22  0.043478         1   46    0
23  0.041667         1   13    0
24  0.040000         1   60    0
25  0.038462         2   55    0
26  0.037037         1  134    0
27  0.035714         2  113    0
28  0.034483 

### TF*IDF-score
* Diskutera hur vi kan använda måtten 
* If similarity is close --> Similar content --> Remove redundance?  



https://forketyfork.medium.com/latex-math-formulas-a-cheat-sheet-21e5eca70aae

In [37]:
# Ranking metric 3 --> TF*IDF

# Term Weights --> Calculate importance of single words in text/doc
# Binary term weights --> document specific
# TF*IDF term weights --> document-collection specific 

# Assign weights to each dimension (attr/word) of each sentence (record/example) 

# Term Frequency (TF-score) --> TFij == frequency of the jth term in in the ith doc 

# Inverse Document Frequency 
# idf-score of the jth term measures the uniqueness of the jth term in the collection of documents
# IDFj = log(M / Nj)
#
# M = total num of docs in collection 
# Nj is the number of documents that contain the jth term

# HIGH TF*IDF-score 
# Word frequent in document && Occur in few documents of the collection 
# LOW TF*IDF-score
# Not present in document || present in all documents of the collection 

### NER 
* (nltk lib) --> (Meningar med Named Entities är troligtvis viktigare)

In [38]:
scores['NER'] = 0
named_entities = named_entity_recognition(org_sentences)
proper_nouns = proper_nouns(org_sentences)
print(proper_nouns)

ner_unique = list(set(named_entities + proper_nouns))
print(ner_unique)
#named_entities = set(named_entities)

# Hittar inte allt p.g.a sentece.split(' ') där den splittar namn som t.ex "ulf kristensson" till "ulf" och "kristensson"
# for i, sentence in enumerate(org_sentences) :
#     #print("i", i)
#     print(i, ": ", sentence)
#     for word in sentence.split(' '):
#         #print("word", word)
#         if word in named_entities:
#             #print(i, word)
#             scores.at[i, "NER"] += 1

#print("org sentences\n", org_sentences)
#print(named_entities)
#scores.describe()
#print(scores)


['SD', 'Socialdemokraternas', 'Lena', 'Aftonbladet', 'KD', 'Rohwedder', 'Aftonbladet', 'Sveriges', 'Kristerssons', 'Lena', 'Hallengren', 'Socialdemokraterna', 'Sverige', 'Sverige', 'Stenevi', 'Putin', 'Biden', 'Sverige', 'Esbati', 'Martin', 'Kinnunen', 'Markus', 'Wiechels', 'Syrien', 'Wiechel', 'Martin', 'Kinnunen', 'UD', 'Syrien', 'Annie', 'KD', 'SD', 'Sverige', 'Nato', 'EU', 'Sveriges', 'Jakob', 'Hallgren']
['Martin', ' Biden', 'Stenevi', ' Syrien', 'Sveriges', ' Twitter', ' Ali', 'Sverige', 'Aftonbladet', 'Wiechel', ' Moderaterna', 'Kinnunen', 'Lena', 'Kristerssons', 'Syrien', ' Putin', ' Markus Wiechels', 'Rohwedder', 'Wiechels', ' Lena Hallengren', ' UD:s', 'Markus', ' KD', 'EU', 'Biden', 'Socialdemokraterna', ' Ulf Kristerssons', 'KD', 'UD', 'SD', 'Socialdemokraternas', ' Aftonbladets My Rohwedder', ' Skämmer', ' Martin Kinnunen', ' Märta', 'Nato', ' Annie', ' Sveriges', 'Annie', ' Aftonbladet', ' Jakob Hallgren', ' Miljöpartiets', ' Nato', 'Putin', ' Sverige', 'Hallengren', 'Jak

In [39]:
# Borrowed from https://stackoverflow.com/questions/33406313/how-to-match-any-string-from-a-list-of-strings-in-regular-expressions-in-python
#p = re.compile(r"\L<words>", words=['fun', 'dum', 'sun', 'gum'])

print(named_entities)
for i, sentence in enumerate(org_sentences):
    print(sentence)
    matches = re.findall(r"(?=(\b" + '|'.join(ner_unique) + r"\b))", sentence) 
    print(matches)
    scores.at[i, "NER"] = len(matches)

['Socialdemokraternas', ' Lena', ' Aftonbladet', ' Moderaterna', ' KD', ' Aftonbladets My Rohwedder', ' Aftonbladet', ' Sveriges', ' Moderaterna', ' Ulf Kristerssons', ' Lena Hallengren', ' Sverige', ' Skämmer', ' Sverige', ' Miljöpartiets', ' Märta', ' Putin', ' Biden', ' Sverige', ' Twitter', ' Ali', ' Esbati', ' Martin Kinnunen', ' Markus Wiechels', ' Syrien', ' Marcus Wiechel', ' Martin Kinnunen', ' UD:s', ' Syrien', ' Twitter', ' Annie', ' KD', ' Sverige', ' Nato', ' Sveriges', ' Jakob Hallgren']
SD får tunga poster i utskotten
['SD']
Publicerad: 30 september Uppdaterad: 30 september
[]
Sverigedemokraterna får ordförandeposten i riksdagens justitie- och utrikesutskott
['Sverige']
Nu går ledarna för vänsterblocket till hård attack
[]
– Det är skrämmande, ganska chockartat, säger Socialdemokraternas gruppledare Lena Hallengren till Aftonbladet
['Socialdemokraterna', ' Lena Hallengren', 'Lena', 'Hallengren', ' Aftonbladet', 'Aftonbladet']
Sverigedemokraterna , Moderaterna, Kristdemok

# Combination Function

#### Standardize

In [40]:
# Standardize
scores_standardized = StandardScaler.fit_transform(self=StandardScaler(), X=scores)
scores_standardized = pd.DataFrame(scores_standardized, columns=columns)
scores_standardized

,Baseline,Headings,TF,NER
0,4.848572,4.663223,-0.566504,-0.460010
1,2.090179,-0.354169,-0.919649,-0.785850
2,1.170715,0.649309,-0.257502,-0.460010
3,0.710982,-0.354169,-0.787219,-0.785850
4,0.435143,-0.354169,-0.463503,1.169192
5,0.251250,1.652788,0.448789,0.191671
6,0.119898,0.649309,-0.949077,-0.134170
7,0.021384,-0.354169,-0.463503,-0.785850
8,-0.055238,-0.354169,-0.331074,-0.785850
9,-0.116536,-0.354169,0.492932,-0.785850


#### Calculate Summarization Length (number of sentences)

In [41]:
num_of_org_sentences = len(org_sentences)
summarization_num_sentences = round(num_of_org_sentences * percentage)

print("summarization: ", summarization_num_sentences, "\noriginal: ", num_of_org_sentences)

summarization:  5 
original:  34


#### Combine
* Combine the scores into one overall score
* add weight and/or ML if time allows

In [42]:
# Combination Function
# Här ligger ML om vi gör det  

final_score = scores_standardized.sum(axis=1)
best_sentences = final_score.nlargest(summarization_num_sentences, keep='all').index.values
print(best_sentences)

print(org_sentences[0])


[ 0 16 32 25 26]
SD får tunga poster i utskotten


# Assemble output 
* Reassemble according to overall score ranking
* Output summarization 

In [43]:
# Assemble Output 
print("Percentage:\n")
for i in best_sentences: 
    print(org_sentences[i])

print("\n\n")

print("N sentences:\n")
for i in best_sentences[0:3]: 
    print(org_sentences[i])

Percentage:

SD får tunga poster i utskotten
Det är tyvärr ett tecken på vart vi är på väg, att för Ulf Kristerssons del är det viktigt att få makt och inflytande, man säljer sig billigt, säger Lena Hallengren, gruppledare för Socialdemokraterna och fortsätter: – Utrikesutskottet är otroligt betydelsefullt med en omvärld som är minst sagt orolig
– Det här är en oerhört viktig post som kanske är mer representativ än andra utskottsposter, men man måste komma ihåg att inom utrikespolitiken har vi en regering, en utrikesminister och en försvarsminister som har mycket mer verkställande makt, säger han och tillägger: – Frågan är hur stort inflytande ordförandefrågorna får om riksdagspartierna förhandlar fram överenskommelser i förväg
Vänsterpartiets riksdagsledamot Ali Esbati lyfter också SD-ledamöterna Martin Kinnunen och Markus Wiechels besök i Syrien 2017 där de mötte Assad-regimen
”För utrikesutskottet antar jag att det står mellan Marcus Wiechel och Martin Kinnunen, här bägge på delegat

In [44]:
# Newspaper Summarization
article.nlp()
print("\nNewspaper3k: \n", article.summary)


Newspaper3k: 
 SD får tunga poster i utskottenPublicerad: 30 september Uppdaterad: 30 septemberSverigedemokraterna får ordförandeposten i riksdagens justitie- och utrikesutskott.
Utrikesutskottet är inte vilket som helst, det ska representera Sveriges riksdag utomlands och överhuvudtaget i relationen till omvärlden, så det är en viktig position.
Den här gången genom att ge Sverigedemokraterna ordförandeposten i utrikesutskottet”, skriver hon på Twitter.
I detta känsliga säkerhetspolitiska läge, när Sverige ska bli medlem i Nato och vara ordförande i EU.
”Får mindre inflytande”Utrikespolitiska institutets direktör Jakob Hallgren tror dock inte att utskottet kommer att ha någon större inverkan på den svenska utrikespolitiken.
